In [1]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

In [3]:
df=pd.read_csv('Cvfile1_val.csv')
#df2=pd.read_csv('obval1.csv')

In [4]:
df

,image_id,file_name,height,width,xmin,xmax,ymin,ymax,category_id,class
0,12,000012.jpg,333,333,155,196,96,174,7,car
1,17,000017.jpg,364,364,184,95,61,138,15,person
2,17,000017.jpg,364,364,89,314,77,259,13,horse
3,23,000023.jpg,500,500,8,237,229,271,2,bicycle
4,23,000023.jpg,500,500,229,105,219,281,2,bicycle
...,...,...,...,...,...,...,...,...,...,...
15657,9958,009958.jpg,333,333,10,211,0,207,15,person
15658,9958,009958.jpg,333,333,1,364,8,324,15,person
15659,9958,009958.jpg,333,333,264,236,1,331,15,person
15660,9958,009958.jpg,333,333,86,243,119,172,15,person


In [5]:
df['class'].unique()

array(['car', 'person', 'horse', 'bicycle', 'aeroplane', 'train',
       'diningtable', 'dog', 'chair', 'cat', 'bird', 'boat',
       'pottedplant', 'tvmonitor', 'sofa', 'motorbike', 'bottle', 'bus',
       'sheep', 'cow'], dtype=object)

In [6]:
bic=df[(df['class'].str.contains('bicycle'))] 
bic=bic.reset_index(drop=True)

In [7]:
bic

,image_id,file_name,height,width,xmin,xmax,ymin,ymax,category_id,class
0,23,000023.jpg,500,500,8,237,229,271,2,bicycle
1,23,000023.jpg,500,500,229,105,219,281,2,bicycle
2,23,000023.jpg,500,500,1,89,177,323,2,bicycle
3,129,000129.jpg,500,500,69,186,201,299,2,bicycle
4,129,000129.jpg,500,500,250,84,241,259,2,bicycle
...,...,...,...,...,...,...,...,...,...,...
413,9667,009667.jpg,375,375,0,105,251,124,2,bicycle
414,9747,009747.jpg,500,500,293,113,0,110,2,bicycle
415,9747,009747.jpg,500,500,357,140,3,173,2,bicycle
416,9761,009761.jpg,500,500,200,11,353,22,2,bicycle


In [8]:
bic['filename']=bic['file_name'].str.replace(r'.jpg','')
bic

<ipython-input-8-55f920232a88>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  bic['filename']=bic['file_name'].str.replace(r'.jpg','')


,image_id,file_name,height,width,xmin,xmax,ymin,ymax,category_id,class,filename
0,23,000023.jpg,500,500,8,237,229,271,2,bicycle,000023
1,23,000023.jpg,500,500,229,105,219,281,2,bicycle,000023
2,23,000023.jpg,500,500,1,89,177,323,2,bicycle,000023
3,129,000129.jpg,500,500,69,186,201,299,2,bicycle,000129
4,129,000129.jpg,500,500,250,84,241,259,2,bicycle,000129
...,...,...,...,...,...,...,...,...,...,...,...
413,9667,009667.jpg,375,375,0,105,251,124,2,bicycle,009667
414,9747,009747.jpg,500,500,293,113,0,110,2,bicycle,009747
415,9747,009747.jpg,500,500,357,140,3,173,2,bicycle,009747
416,9761,009761.jpg,500,500,200,11,353,22,2,bicycle,009761


In [9]:
bic['ano'] = bic['xmin'].map(str) + ' ' + bic['ymin'].map(str) + ' ' + bic['xmax'].map(str) + ' ' + bic['ymax'].map(str)
bic 

,image_id,file_name,height,width,xmin,xmax,ymin,ymax,category_id,class,filename,ano
0,23,000023.jpg,500,500,8,237,229,271,2,bicycle,000023,8 229 237 271
1,23,000023.jpg,500,500,229,105,219,281,2,bicycle,000023,229 219 105 281
2,23,000023.jpg,500,500,1,89,177,323,2,bicycle,000023,1 177 89 323
3,129,000129.jpg,500,500,69,186,201,299,2,bicycle,000129,69 201 186 299
4,129,000129.jpg,500,500,250,84,241,259,2,bicycle,000129,250 241 84 259
...,...,...,...,...,...,...,...,...,...,...,...,...
413,9667,009667.jpg,375,375,0,105,251,124,2,bicycle,009667,0 251 105 124
414,9747,009747.jpg,500,500,293,113,0,110,2,bicycle,009747,293 0 113 110
415,9747,009747.jpg,500,500,357,140,3,173,2,bicycle,009747,357 3 140 173
416,9761,009761.jpg,500,500,200,11,353,22,2,bicycle,009761,200 353 11 22


In [10]:
# here all the images of bicyle have been seperated to bicycleimages folder
import shutil, os
os.chdir('F:\\DATA SETS\\VOCtrainval_06-Nov-2007\\VOCdevkit\\VOC2007\\JPEGImages')
for i in bic['file_name']:
    shutil.copy(i,'bicycleimages') 
#shutil-method in Python is used to copy the content of the source file to the destination file or directory.

In [11]:
# here all the anotations of bicyle have been seperated to bicano folder in csv format 
#(crete empty folder and give path..all came in csv formate)--it convert sep. all jpg. file and csv for individual anotations 
os.chdir('C:\\Users\\Amit\\Data Science\\Projects\\Project - 2 CV\\Object Detection Project\\bicano')
for (filename), group in bic['ano'].groupby(bic['filename']):
    group.to_csv(f'{filename}.csv', index=False)

#upto above steps i have done in jupyter notebook
# then rest i have done in colab by uploading folders in google drive

In [23]:
#seprated bicyleimages and anno file we upload in drive b'coz this files directly connected in drive-colab connector

In [24]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [21]:
cv2.__version__

'4.5.4'

In [15]:
#pip uninstall opencv-contrib-python opencv-python


In [16]:
#pip uninstall opencv-contrib-python

In [17]:
#pip install opencv-contrib-python


In [22]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

AttributeError: module 'cv2' has no attribute 'ximgproc'

In [ ]:
annot = "/content/drive/MyDrive/busano"
path='/content/drive/MyDrive/busimages'
train_images=[]
train_labels=[]
for e,i in enumerate(os.listdir(annot)):
    try:
        if i.startswith("0"):
            filename = i.split(".")[0]+".jpg"
            print(e,filename)
            image = cv2.imread(os.path.join(path,filename))
        df = pd.read_csv(os.path.join(annot,i))
        gtvalues=[]
        for row in df.iterrows():
            x1 = int(row[1][0].split(" ")[0])
            y1 = int(row[1][0].split(" ")[1])
            x2 = int(row[1][0].split(" ")[2])
            y2 = int(row[1][0].split(" ")[3])
            gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
        ss.setBaseImage(image)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = image.copy()
        counter = 0
        falsecounter = 0
        flag = 0
        fflag = 0
        bflag = 0
        for e,result in enumerate(ssresults):
            if e < 2000 and flag == 0:
                for gtval in gtvalues:
                    x,y,w,h = result
                    iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                    if counter < 30:
                        if iou > 0.60:
                            timage = imout[y:y+h,x:x+w]
                            resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                            train_images.append(resized)
                            train_labels.append(1)
                            counter += 1
                    else :
                        fflag =1
                    if falsecounter <30:
                        if iou < 0.4:
                            timage = imout[y:y+h,x:x+w]
                            resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                            train_images.append(resized)
                            train_labels.append(0)
                            falsecounter += 1
                    else :
                        bflag = 1
                if fflag == 1 and bflag == 1:
                    print("inside")
                    flag = 1
    except Exception as e:
        print(e)
        print("error in "+filename)
        continue

In [ ]:
train_images

In [ ]:
len(train_labels)

In [ ]:
train_labels

In [ ]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

In [ ]:
X_new.shape

In [ ]:
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

In [ ]:
for layers in (vggmodel.layers)[:15]:
    print(layers)
    layers.trainable = False
X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(inputs= vggmodel.input, outputs= predictions)
#from keras.optimizers import adam
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.0001)
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_final.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)
lenc = MyLabelBinarizer()
Y =  lenc.fit_transform(y_new)
X_train, X_test , y_train, y_test = train_test_split(X_new,Y,test_size=0.10)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
trdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
traindata = trdata.flow(x=X_train, y=y_train)
tsdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
testdata = tsdata.flow(x=X_test, y=y_test)
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 10, epochs= 10, validation_data= testdata, validation_steps=2, callbacks=[checkpoint,early])
import matplotlib.pyplot as plt
# plt.plot(hist.history["acc"])
# plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss","Validation Loss"])
plt.show()
plt.savefig('chart loss.png')

In [ ]:
len(X_test)

In [ ]:
X_test[328]

In [ ]:
im = X_test[10]
plt.imshow(im)
img = np.expand_dims(im, axis=0)
out= model_final.predict(img)
if out[0][0] > out[0][1]:
    print("bus")
else:
    print("no bus")
z=0
for e,i in enumerate(os.listdir(path)):
    if i.startswith("4"):
        z += 1
        img = cv2.imread(os.path.join(path,i))
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = img.copy()
        for e,result in enumerate(ssresults):
            if e < 2000:
                x,y,w,h = result
                timage = imout[y:y+h,x:x+w]
                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)
                out= model_final.predict(img)
                if out[0][0] > 0.65:
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
        plt.figure()
        plt.imshow(imout)

In [ ]:
im = X_test[3]
plt.imshow(im)
img = np.expand_dims(im, axis=0)
out= model_final.predict(img)
if out[0][0] > out[0][1]:
    print("bus")
else:
    print("no bus")
z=0
for e,i in enumerate(os.listdir(path)):
    if i.startswith("4"):
        z += 1
        img = cv2.imread(os.path.join(path,i))
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = img.copy()
        for e,result in enumerate(ssresults):
            if e < 2000:
                x,y,w,h = result
                timage = imout[y:y+h,x:x+w]
                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)
                out= model_final.predict(img)
                if out[0][0] > 0.65:
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
        plt.figure()
        plt.imshow(imout)